# Le machine learning

- Machine Learning : Techniques de calcul permettant aux ordinateurs d’apprendre à exécuter des tâches en s’inspirant de patterns perçus sur des données, avec peu d’intervention humaine. Les tâches se résument souvent à de la prédiction.

![image](./images/ml1.png)
    

# Quelques exemples

- Prestation de service : identification des paramètres de prédisposition à la résiliation (churn)
- Marketing: identification de segments distincts de consommateurs 
- Production : identification de paramètres induisant des défaillances
- Pharmaceutique: classification de médicaments selon le profil chimique


# Le data mining vs le machine learning

- Data Mining : recherche d’information et de patterns cachés dans une base de données isolée ou en croisant plusieurs bases. C’est l’être humain qui explore et prend des décisions.
- Le data mining s’appuie sur des techniques et algorithmes issus de la statistique et de la gestion de bases de données.
- Les données exploitées ont souvent une taille considérable.


![image](./images/trend-ml.png)

# Les outils

![image](./images/poll-ml.png)

# Les types de machine learning

- Les algorithmes de machine learning peuvent être rassemblés dans 3 groupes principaux :
   - L'apprentissage supervisé
     - Cible (variable dépendante qualitative ou quantitative)
     - Prédicteurs (variables indépendantes)
     - Régression, arbre de decision, foret aléatoire, kNN, regression logistique, SVM…

   - L'apprentissage non supervisé
     - Pas de variable dépendante, pas de modélisation
     - Classer des individus dans différents groups ou trouver des associations
     - Apriori algorithm, K-means.

   - L'apprentissage par renforcement
     - Apprentissage pour prendre des décisions spécifique afin d’aboutir à un but précis
     - Markov Decision Process

# L’analyse de données et le machine learning avec Python

Le package la plus adaptée pour faire du data mining et du machine learning est la package **scikit-learn**


Ce package propose des fonctions prédéfinies pour un grand nombre de méthodes

- La classification : SVM, plus proches voisins, random forest…
- Les régressions : linéaire, ridge, Lasso…
- Le clustering : k-means…
- L’analyse de données :  ACP, DA…

Dans le cadre de cette formation, l'objectf n'est pas de décrire la théorie des méthodes mais plutôt de comprendre l'utilisation de Python pour les appliquer.

# L’apprentissage supervisé avec Scikit-Learn
Les méthodes d’apprentissage supervisé sont les méthodes actuellement les plus
utilisées en data science. Il s’agit d’essayer de prédire une variable cible et d’utiliser
différentes méthodes pour arriver à cette fin.
Nous allons illustrer ces méthodes de traitement de données avec du code et des
cas pratiques.

### Les données et leur transformation
Ce jeu de données est composé de 3333 individus et de 18 variables. Il est stocké dans un fichier csv, nommé telecom.csv, accessible dans le répertoire Data. On le récupère en utilisant Pandas :

In [1]:
import pandas as pd
import numpy as np

In [2]:
churn=pd.read_csv("./data/telecom.csv")

In [3]:
churn.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


## Les données

Ce jeu de données n’a pas de données manquantes et nous allons devoir effectuer
quelques transformations pour l’adapter à nos traitements. Nous voyons par exemple qu’il est composé de trois colonnes object.

Nous pouvons afficher les statistiques descriptives pour les colonnes object :

In [4]:
churn.describe(include="object").transpose()

,count,unique,top,freq
State,3333,51,WV,106
Phone,3333,3333,401-5915,1
Int'l Plan,3333,2,no,3010
VMail Plan,3333,2,no,2411
Churn?,3333,2,False.,2850


## Transformation des données

On voit que les données sont toutes binaires. 

Pour les variables binaires, il nous suffit de les recoder avec Scikit-Learn pour obtenir des données exploitables. Par
ailleurs, il existe une autre variable qualitative dans notre jeu de données, Area Code,
qui est numérique mais avec trois modalités :

In [5]:
churn["Area Code"].value_counts()

415    1655
510     840
408     838
Name: Area Code, dtype: int64

In [6]:
churn = pd.concat([churn,pd.get_dummies(churn["Area Code"])], axis=1)

In [7]:
churn.columns

Index([         'State', 'Account Length',      'Area Code',          'Phone',
           'Int'l Plan',     'VMail Plan',  'VMail Message',       'Day Mins',
            'Day Calls',     'Day Charge',       'Eve Mins',      'Eve Calls',
           'Eve Charge',     'Night Mins',    'Night Calls',   'Night Charge',
            'Intl Mins',     'Intl Calls',    'Intl Charge', 'CustServ Calls',
               'Churn?',              408,              415,              510],
      dtype='object')

In [8]:
pd.get_dummies(churn["Area Code"]).columns

Int64Index([408, 415, 510], dtype='int64')

## La préparation des données

Nous allons utiliser le processus de traitement classique pour transformer nos
données avec Scikit-Learn. Dans ce cas, nous n’avons pas de données manquantes,
nous travaillons donc sur la transformation des variables qualitatives.

In [9]:
# on définit les colonnes
# les colonnes quantitatives
col_quanti = ['Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls',408, 415, 510]

## Prédire l’attrition des clients
Lorsqu’on veut prédire une variable binaire, on devra avoir une colonne du type
binaire. On préfère généralement un codage 0/1 afin de garder un type entier simple à gérer. 

Les variables explicatives x auront été préparées de manière intelligente afin de bien appliquer nos modèles.

On crée donc x et y :

In [10]:
x = churn[col_quanti]
y = churn["Churn?"]

## Séparation des données

Pour la séparation, on utilise la fonction train_test_split() de Scikit-Learn.

Cette fonction permet de créer automatiquement autant de structures que nécessaire
à partir de nos données. 

Elle utilise une randomisation des individus et ensuite une séparation en fonction d’un paramètre du type test_size :

In [11]:
# on importe la fonction
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Dans certains cas, il peut arriver qu’il y ait une forte disparité de distribution des
modalités entre les proportions d’acceptation et de refus. On peut vouloir faire en
sorte que les répartitions des modalités de y soient égales dans les différents échantillons,
on pourra alors utiliser une stratification. On va utiliser une stratification en
prenant y comme base pour effectuer la stratification :

In [12]:
assert x_train.shape[0] == y_train.shape[0]
print("Bien séparé !")

Bien séparé !


In [13]:
x_train.shape

(2666, 16)

In [14]:
y_train.shape

(2666,)

Ainsi les deux échantillons _train et _test ont la même distribution

## Le choix et l’ajustement de l’algorithme

Tout au long de ce Notebook, nous allons essayer d'ajouter un nouveau modèle, il s'agit du modèle GBM
```python
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_models import LogisticRegression
```

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
# ajouter le 3ème modèle

Ensuite, on crée un objet à partir de la classe du modèle en lui fournissant les
hyperparamètres dont il a besoin :

In [17]:
modele_rf = RandomForestClassifier(n_estimators=100)
modele_knn = KNeighborsClassifier(n_neighbors=5)
modele_logit = LogisticRegression(solver = 'newton-cg')
modele_gbm = GradientBoostingClassifier()

Dans ce cas, on prend les hyperparamètres par défaut.

On peut ensuite ajuster notre modèle en utilisant les données :

In [18]:
modele_rf.fit(x_train,y_train)
modele_knn.fit(x_train,y_train)
modele_gbm.fit(x_train,y_train)
modele_logit.fit(x_train,y_train)
# ajouter le 3ème modèle
pass

Une fois qu’on a estimé les paramètres du modèle, on va pouvoir extraire des
informations. De nouveaux attributs de chaque classe apparaissent, ils se terminent par le symbole underscore _ :

In [19]:
modele_rf.n_features_in_

16

In [20]:
pd.DataFrame(modele_rf.feature_importances_,index=x_train.columns)

,0
Day Mins,0.151697
Day Calls,0.047841
Day Charge,0.154002
Eve Mins,0.080780
Eve Calls,0.044598
Eve Charge,0.084244
Night Mins,0.052132
Night Calls,0.051989
Night Charge,0.051986
Intl Mins,0.049556


Ce qui va nous intéresse avant tout, c’est de prédire avec notre modèle. Pour cela nous allons utiliser la méthode .predict() :

In [21]:
y_predict_rf = modele_rf.predict(x_test)
y_predict_knn = modele_knn.predict(x_test)
y_predict_gbm = modele_gbm.predict(x_test)
y_predict_logit = modele_logit.predict(x_test)

In [22]:
(y_predict_rf == y_test).sum()/y_test.shape[0]

0.8995502248875562

In [23]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, roc_curve

dico_model = { "rf" : RandomForestClassifier(),
             "gbm" : GradientBoostingClassifier(),
             "logit" : LogisticRegression(solver = 'newton-cg'),
             "knn" : KNeighborsClassifier()}

for model in dico_model.keys():
    model_temp = dico_model[model]
    model_temp.fit(x_train, y_train)
    y_predict = model_temp.predict(x_test)
    accuracy_modele = accuracy_score(y_test,y_predict)
    print(f"Pourcentage de bien classés pour le modèle {model} : {accuracy_modele}")
    confusion_matrix_model=confusion_matrix(y_test,y_predict)
    print(f"Matrice de confusion pour le modèle {model} :",
          confusion_matrix_model, sep="\n")


Pourcentage de bien classés pour le modèle rf : 0.9070464767616192
Matrice de confusion pour le modèle rf :
[[553  13]
 [ 49  52]]
Pourcentage de bien classés pour le modèle gbm : 0.8935532233883059
Matrice de confusion pour le modèle gbm :
[[549  17]
 [ 54  47]]
Pourcentage de bien classés pour le modèle logit : 0.848575712143928
Matrice de confusion pour le modèle logit :
[[561   5]
 [ 96   5]]
Pourcentage de bien classés pour le modèle knn : 0.8650674662668666
Matrice de confusion pour le modèle knn :
[[548  18]
 [ 72  29]]


On obtient ainsi une valeur prédite pour les éléments de notre échantillon de
validation.

## Les indicateurs pour valider un modèle
La partie validation d’un modèle d’apprentissage supervisé est extrêmement
importante. L’objectif d’un modèle d’apprentissage supervisé est de prédire une
valeur la plus proche possible de la réalité. Nous différencions trois types d’indices
en fonction du type de variable cible. Tous les indicateurs de qualité du modèle sont
stockés dans le module *metrics* de Scikit-Learn.

## Le pourcentage de bien classés
Il s’agit de l’indicateur le plus connu. On le nomme accuracy. Il est calculé à partir du rapport entre le nombre d’individus bien classés et le nombre total d’individus dans l’échantillon.

In [24]:
from sklearn.metrics import accuracy_score, recall_score

accuracy_modele_rf = accuracy_score(y_test,y_predict_rf)
accuracy_modele_knn = accuracy_score(y_test,y_predict_knn)
print("Pourcentage de bien classés pour le modèle RF : %.3f" %(accuracy_modele_rf))
print("Pourcentage de bien classés pour le modèle kNN :%.3f" %(accuracy_modele_knn))


Pourcentage de bien classés pour le modèle RF : 0.900
Pourcentage de bien classés pour le modèle kNN :0.865


## La matrice de confusion
Il s’agit d’un autre indicateur important pour juger de la qualité d’un modèle, il n’est pas défini par une seule valeur mais par une matrice dans laquelle on peut lire le croisement entre les valeurs observées et les valeurs prédites à partir du modèle. 

Pour calculer cette matrice, on pourra utiliser :

In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix_rf=confusion_matrix(y_test,y_predict_rf)
confusion_matrix_knn=confusion_matrix(y_test,y_predict_knn)
print("Matrice de confusion pour le modèle RF :",
confusion_matrix_rf, sep="\n")
print("Matrice de confusion pour le modèle kNN :",
confusion_matrix_knn, sep="\n")
# ajouter le 3ème modèle

Matrice de confusion pour le modèle RF :
[[554  12]
 [ 55  46]]
Matrice de confusion pour le modèle kNN :
[[548  18]
 [ 72  29]]


## L’aire sous la courbe ROC
La courbe ROC est un indicateur important mais on préfère souvent une valeur plutôt
qu’une courbe afin de comparer nos modèles. Pour cela, on utilise l’aire sous la courbe
ROC (AUC). Cette aire est calculée directement à partir de la courbe ROC. Ainsi, un
modèle aléatoire aura une AUC de 0.5 et un modèle parfait aura une AUC de 1.

In [26]:
from sklearn.metrics import roc_auc_score
auc_modele_rf=roc_auc_score(y_test, modele_rf.predict_proba(x_test)[:,1])
auc_modele_knn=roc_auc_score(y_test,modele_knn.predict_proba(x_test)[:,1])

print("Aire sous la courbe ROC pour le modèle RF :" ,auc_modele_rf)
print("Aire sous la courbe ROC pour le modèle kNN :" ,auc_modele_knn)
# ajouter le 3ème modèle

Aire sous la courbe ROC pour le modèle RF : 0.872380435923451
Aire sous la courbe ROC pour le modèle kNN : 0.7092852394780114


## La validation croisée
Jusqu’ici nous avons utilisé des indicateurs basés sur une seule occurrence de test. Ceci veut dire qu’on ne teste notre modèle que sur un seul échantillon.

Une approche alternative souvent utilisée est la validation croisée. Celle-ci est en fait basée sur la répétition de l’estimation et de la validation sur des données différentes.

Pour obtenir ce cv-score, on utilise :

In [27]:
from sklearn.model_selection import cross_val_score

scores_rf = cross_val_score(modele_rf, x_train, y_train, cv=5, scoring='roc_auc')
scores_knn = cross_val_score(modele_knn, x_train, y_train, cv=5, scoring='roc_auc')

print("AUC pour RF : %.3f (+/- %.3f)"% (scores_rf.mean(), scores_rf.std() * 2))
print("AUC pour kNN : %.3f (+/- %.3f)"% (scores_knn.mean(),scores_knn.std() * 2))
# ajouter le 3ème modèle

AUC pour RF : 0.854 (+/- 0.058)
AUC pour kNN : 0.637 (+/- 0.084)


## L’ajustement des hyperparamètres d’un modèle

L’une des tâches du data scientist est de trouver le meilleur modèle possible. La
plupart des modèles de machine learning ont des hyperparamètres. Il s’agit de paramètres
du modèle qui sont définis en amont de l’ajustement.

Scikit-Learn propose une classe GridSearchCV permettant d’implémenter cette
recherche d’hyperparamètres :

In [28]:
from sklearn.model_selection import GridSearchCV

On va donc devoir définir les hyperparamètres que l’on souhaite tester. Pour cela,
on utilisera un dictionnaire d’hyperparamètres, par exemple :

In [29]:
dico_param= {"max_depth":[3,5,7,10], "n_estimators":[10,20,50,100,500]}

On va encore utiliser l’accuracy pour valider notre modèle. Finalement, nous allons
utiliser une validation croisée à cinq groupes pour valider les résultats.
Le nouvel objet est le suivant :

In [30]:
recherche_hyper = GridSearchCV(RandomForestClassifier(), 
                               dico_param, 
                               scoring="accuracy",
                               cv=5)

Une fois qu’on a créé cet objet, on peut lui joindre les données afin d’estimer les
meilleurs paramètres du modèle.

Cette étape peut être très longue.

In [32]:
recherche_hyper.fit(x_train, y_train)
pass

In [33]:
print(recherche_hyper.best_params_)

{'max_depth': 10, 'n_estimators': 500}


In [34]:
print(recherche_hyper.best_score_)

0.9096057226777973


In [35]:
pd.DataFrame(recherche_hyper.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.035281,0.002627,0.004731,0.000094,3,10,"{'max_depth': 3, 'n_estimators': 10}",0.870787,0.878049,0.861163,0.861163,0.872420,0.868716,0.006621,19
1,0.060792,0.000631,0.005795,0.000072,3,20,"{'max_depth': 3, 'n_estimators': 20}",0.867041,0.878049,0.864916,0.863039,0.868668,0.868343,0.005213,20
2,0.145068,0.002299,0.009113,0.000108,3,50,"{'max_depth': 3, 'n_estimators': 50}",0.863296,0.874296,0.868668,0.866792,0.872420,0.869094,0.003930,16
3,0.289925,0.005813,0.014935,0.000446,3,100,"{'max_depth': 3, 'n_estimators': 100}",0.865169,0.872420,0.872420,0.861163,0.872420,0.868719,0.004707,17
4,1.426708,0.049041,0.057284,0.002189,3,500,"{'max_depth': 3, 'n_estimators': 500}",0.867041,0.872420,0.868668,0.866792,0.868668,0.868718,0.002012,18
5,0.040788,0.000546,0.004447,0.000078,5,10,"{'max_depth': 5, 'n_estimators': 10}",0.895131,0.891182,0.898687,0.893058,0.894934,0.894598,0.002496,15
6,0.076506,0.000990,0.005660,0.000073,5,20,"{'max_depth': 5, 'n_estimators': 20}",0.880150,0.908068,0.900563,0.898687,0.900563,0.897606,0.009304,14
7,0.182992,0.002177,0.009465,0.000112,5,50,"{'max_depth': 5, 'n_estimators': 50}",0.891386,0.904315,0.908068,0.889306,0.900563,0.898727,0.007273,12
8,0.360383,0.006048,0.015290,0.000373,5,100,"{'max_depth': 5, 'n_estimators': 100}",0.889513,0.906191,0.902439,0.896811,0.896811,0.898353,0.005675,13
9,1.760322,0.007634,0.060055,0.000572,5,500,"{'max_depth': 5, 'n_estimators': 500}",0.889513,0.908068,0.908068,0.893058,0.906191,0.900980,0.008023,9


**Exercice :**

Effectuez le même travail avec GBM

In [36]:
recherche_hyper_gbm = ___


In [37]:
dico_param= {"max_depth":[3,5,7,10], "n_estimators":[10,20,50,100],"learning_rate":[0.01,0.1,0.4 ]}

On va encore utiliser l’accuracy pour valider notre modèle. Finalement, nous allons
utiliser une validation croisée à cinq groupes pour valider les résultats.
Le nouvel objet est le suivant :

In [38]:
recherche_hyper_gbm = GridSearchCV(GradientBoostingClassifier(), 
                               dico_param, 
                               scoring="accuracy",
                               cv=5)

Une fois qu’on a créé cet objet, on peut lui joindre les données afin d’estimer les
meilleurs paramètres du modèle.

Cette étape peut être très longue.

In [39]:
recherche_hyper_gbm.fit(x_train, y_train)
pass

In [40]:
print(recherche_hyper_gbm.best_params_)

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}


In [41]:
print(recherche_hyper_gbm.best_score_)

0.9036026730189514


In [42]:
pd.DataFrame(recherche_hyper_gbm.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.081331,0.000300,0.003536,0.000117,0.01,3,10,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",0.855805,0.857411,0.857411,0.857411,0.855535,0.856715,0.000857,39
1,0.164177,0.002325,0.003934,0.000212,0.01,3,20,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",0.855805,0.857411,0.857411,0.857411,0.855535,0.856715,0.000857,39
2,0.392180,0.009098,0.003932,0.000035,0.01,3,50,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",0.855805,0.857411,0.857411,0.857411,0.855535,0.856715,0.000857,39
3,0.769917,0.004501,0.004347,0.000092,0.01,3,100,"{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",0.878277,0.894934,0.900563,0.893058,0.893058,0.891978,0.007382,22
4,0.131942,0.000469,0.003712,0.000066,0.01,5,10,"{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",0.855805,0.857411,0.857411,0.857411,0.855535,0.856715,0.000857,39
5,0.270073,0.006672,0.004610,0.001055,0.01,5,20,"{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",0.855805,0.857411,0.857411,0.857411,0.855535,0.856715,0.000857,39
6,0.654643,0.010039,0.005035,0.001219,0.01,5,50,"{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",0.855805,0.857411,0.857411,0.857411,0.859287,0.857465,0.001103,37
7,1.284532,0.003653,0.005025,0.000101,0.01,5,100,"{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",0.889513,0.900563,0.896811,0.906191,0.893058,0.897227,0.005806,12
8,0.190668,0.005250,0.004111,0.000164,0.01,7,10,"{'learning_rate': 0.01, 'max_depth': 7, 'n_est...",0.855805,0.857411,0.857411,0.857411,0.855535,0.856715,0.000857,39
9,0.384367,0.003525,0.004343,0.000097,0.01,7,20,"{'learning_rate': 0.01, 'max_depth': 7, 'n_est...",0.855805,0.857411,0.857411,0.857411,0.855535,0.856715,0.000857,39


## Passer en production votre modèle d’apprentissage supervisé

### Persistance de modèle avec Scikit-Learn

Python possède plusieurs outils pour la persistance d’objets, c’est-à-dire pour stocker
des objets dans des fichiers. Les objets de Scikit-Learn sont aussi dans cette
situation. On utilise un format pickle qui aura l’extension .pkl.

Par exemple, si nous voulons sauvegarder le dernier pipeline de traitement, nous
allons utiliser :

In [43]:
import joblib
joblib.dump(recherche_hyper_gbm, './modele_grid_gbm.pkl')

['./modele_grid_gbm.pkl']

Une fois ce modèle stocké, on peut très bien le réutiliser dans un autre cadre. Si
nous créons un nouveau notebook, nous allons utiliser :



In [44]:
import joblib
#grid_search_mon_pipe = joblib.load('./data/modele_grid_pipe.pkl')

On peut ensuite appliquer le modèle avec tous les paramètres qui ont été appris :


```python
grid_search_mon_pipe.predict(x_test)
```

L’utilisation d’un fichier Pickle dans un notebook est une technique assez simple et courante.